In [ ]:
# ================================
# Protein Sequence Similarity Calculation - Smith-Waterman (SIMD-accelerated)
# Using Parasail (SIMD, Striped) for exact SW + affine gap, BLOSUM62
# Designed for Google Colab
# ================================

# Step 1: Install dependencies
print("Installing required dependencies...")
!pip -q install parasail biopython tqdm
print("Dependencies installed!\n")

# Step 2: Import required libraries (Parasail for SW computation)
from google.colab import files
import parasail
from tqdm import tqdm
from itertools import combinations

# Step 3: Upload protein file
print("=" * 60)
print("Please upload your protein file.")
print("Example format: index\\tUniProt\\tName\\tSequence  (4 columns)")
print("If your file uses a different number of columns, it will still be parsed:")
print("  - First column = ID")
print("  - Last column = sequence")
print("=" * 60)
uploaded = files.upload()

# Step 4: Parse protein file (robust: tab-delimited preferred, otherwise whitespace)
def parse_protein_file(file_content):
    """Parse the protein file and return a dictionary {ID: sequence}."""
    proteins = {}
    text = file_content.decode('utf-8', errors='ignore')
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]

    for ln in lines:
        parts = ln.split('\t')
        if len(parts) < 2:
            parts = ln.split()  # fallback: whitespace split
        if len(parts) >= 2:
            prot_id = parts[0].strip()
            seq = parts[-1].strip().upper()
            # Keep only alphabetic characters
            seq = ''.join([c for c in seq if c.isalpha()])
            if prot_id and seq:
                proteins[prot_id] = seq
    return proteins

# Parse uploaded file
filename = list(uploaded.keys())[0]
file_content = uploaded[filename]
proteins = parse_protein_file(file_content)

print(f"\nSuccessfully loaded {len(proteins)} protein sequences.")
print("Example protein IDs:", list(proteins.keys())[:10])

# Step 5: Compute normalized similarity using Parasail SW (BLOSUM62 + affine gap)
# Scoring matrix and gap penalties
matrix = parasail.blosum62
GAP_OPEN = 10     # gap open penalty (positive integer, internally used as -open)
GAP_EXT  = 1      # gap extension penalty (positive integer, internally as -extend)

# Use trace-free striped SW (fast and exact score)
def sw_score(seq1, seq2):
    """Return Smith–Waterman alignment score (BLOSUM62, affine gap) using Parasail."""
    res = parasail.sw_striped_16(seq1, seq2, GAP_OPEN, GAP_EXT, matrix)
    return res.score

def calculate_normalized_similarity(seq1, seq2, self_scores):
    """
    Compute normalized similarity:
        SW_score / sqrt( self(seq1) * self(seq2) )
    Range: [0, 1]
    """
    score12 = sw_score(seq1, seq2)
    denom = (self_scores[seq1] * self_scores[seq2]) ** 0.5
    if denom <= 0:
        return 0.0
    val = score12 / denom
    # Clamp to the interval [0, 1]
    if val < 0:
        val = 0.0
    if val > 1:
        val = 1.0
    return val

# Precompute self-alignment scores
print("\nPrecomputing self-alignment scores (self-score, Parasail SW)...")
self_scores = {}
for pid, seq in tqdm(proteins.items(), total=len(proteins)):
    self_scores[seq] = sw_score(seq, seq)
print("Self-score computation completed!\n")

# Step 6: Pairwise similarity computation
print("\nStarting pairwise similarity computation (SIMD SW, BLOSUM62)...")
THRESHOLD = 0.30
print(f"Similarity threshold: >= {THRESHOLD:.2f}")
print("=" * 60)

results = []
protein_ids = list(proteins.keys())
pairs_iter = combinations(protein_ids, 2)
total_pairs = len(protein_ids) * (len(protein_ids) - 1) // 2

count = 0
filtered_count = 0
for id1, id2 in tqdm(pairs_iter, total=total_pairs, desc="Pairwise SW"):
    count += 1
    seq1 = proteins[id1]
    seq2 = proteins[id2]

    sim = calculate_normalized_similarity(seq1, seq2, self_scores)

    if sim >= THRESHOLD:
        results.append(f"{id1}\t{id2}\t{sim:.4f}")
        filtered_count += 1

print("=" * 60)
print("Computation completed!")
print(f"Total sequence pairs evaluated: {count}")
print(f"Pairs with similarity >= {THRESHOLD:.2f}: {filtered_count}\n")

# Step 7: Save and download results
if len(results) == 0:
    print("⚠️ Warning: No protein pairs found with similarity >= threshold.")
    print("Suggestion: Try lowering the threshold or check your input file.")
else:
    output_filename = "protein_similarity_results.txt"
    header = "Protein_ID_1\tProtein_ID_2\tNormalized_Similarity\n"

    with open(output_filename, 'w') as f:
        f.write(header + "\n".join(results))

    print(f"Results saved to: {output_filename}")
    print(f"Similarity range: {THRESHOLD:.2f} - 1.0 (1.0 indicates highly similar/identical local regions)")
    print("\nPreview of the first 5 results:")
    print("=" * 60)
    print(header.strip())
    for line in results[:5]:
        print(line)
    if len(results) > 5:
        print(f"... ({len(results)-5} more entries)")
    print("=" * 60)

    print("\nPreparing download...")
    files.download(output_filename)
    print("Download completed!")
